# Style Guide

The code style should follow the [PEP 8](https://www.python.org/dev/peps/pep-0008/) guidelines. 

The code should be formatted using
[black](https://black.readthedocs.io/en/stable/). 

We are not close to lint-compliance yet, but we are working on it.

## Complying with Naming Conventions

Beyond what is defined in the PEP 8 guidelines, we have the following naming conventions:

- **Minimize the use of abbreviations.** If an abbreviation is used, it should be well-known and non-ambiguous.

- **Use standard names for classes.** Use the following names:
  - `layer` for a class that represents a single layer in a neural network, typically the learnable part of a block.
  - `activation` for a class that represents a non-learnable activation function.
  - `normalization` for a class that represents a normalization layer.
  - `dropout` for a class that represents a dropout layer.
  - `pool` for a class that represents a pooling layer.
  - `upsample` for a class that represents an upsampling layer.
  - `block` / `blocks` for a class that represents a block in a neural network, typically a sequence of layers.
  - `backbone` for a class that represents the main part of a neural network, typically a sequence of blocks.
  - `head` for a class that represents the final part of a neural network, typically a single layer followed by an optional activation function.
  - `model` for a class that represents a full neural network architecture.
  - `optimizer` for a class that represents an optimizer.
  - `loss` for a class that represents a loss function.
  - `metric` for a class that represents a metric.
  
- **Use underscored numbers when necessary.** If there is a naming conflict within a class, add numbers to the end of the name with an underscore, 0-indexed. For example, `layer_0` and `layer_1`.
  - This is correct: `layer_0`, `layer_1`, `layer_2`
  - This is incorrect: ~~`layer_1`, `layer_2`, `layer_3`~~
  - This is incorrect: ~~`layer`, `layer_1`, `layer_3`~~

## Using Type Hints

**Use type hints extensively.** This will make the code more readable and maintainable.

The type hints should be as specific as possible.
For example, if a string can be one of several values, use a `Literal` type hint. 
Similarly, if a function takes a list of integers, the type hint should be `List[int]` instead of `List`. 

We are currently supporting Python 3.8 and above. Some features of Python 3.9 are not supported yet, such as the `|` operator for type hints. You can get around this by importing `annotations` from `__future__`.

Classes should have their attribute types defined before the `.__init__()` method. An example is shown below:

```python
class MyClass:
    """A class that does something."""
    attribute: int

    def __init__(self, num: int):
        """Initialize class"""
        self.attribute = num
```

## Formatting Imports

**Use absolute imports.** This should be done for all imports, unless the target is at the same level in the hierarchy inside a `__init__.py`.
For example, in `deeplay/external/optimizers/optimizer.py`:
```python
from deeplay.module import DeeplayModule  # Correct.
from ...module import DeeplayModule  # Incorrect.

from deeplay.external.optimizers.adam import Adam  # Correct.
from .adam import Adam #  Also correct.
from deeplay.external.optimizers.adam import *  # Incorrect.
```

In a `__init__.py` file, you may use * imports from directories, but not from files. From files, you should import the specific classes or functions you need.
For example, in `deeplay/external/__init__.py`:
```python
from .optimizers import *  # Correct.
from .optimizers.adam import Adam  # Correct.
from .optimizers.adam import *  # Incorrect.
```

## Writing Documentation

Documentation should follow the [NumpyDoc style guide](https://numpydoc.readthedocs.io/en/latest/format.html#style-guide).

In general, all non-trivial classes and methods should be documented. The documentation should include a description of the class or method, the parameters, the return value, and any exceptions that can be raised. We sincerely appreciate any effort to improve the documentation, particularly by including examples of how to use the classes and methods.

### Writing Documentation for Functions and Methods

The following describes how a function or method should be documented ...

In [1]:
from typing import List

def my_function(param1: int, param2: str) -> List[int]:
    """This is a short description of the function on one line.

    This is a longer description of the function. It should explain what the 
    function does and how it works. Note also that each line is no longer than 
    79 characters.

    Parameters
    ----------
    param1 : int
        This is a description of the first parameter.
    param2 : str
        This is a description of the second parameter.

    Returns
    -------
    list of int
        This is a description of the return value.

    Examples
    --------
    >>> my_function(1, 'a')
    [1, 2, 3]

    Raises
    ------
    ValueError
        This is a description of the exception that can be raised.
    """
    ...

... and the following is a more concrete example of a documented function.

In [2]:
import torch
from typing import Union, Tuple

def predict(
        self, x, *args, batch_size=32, device=None, output_device=None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, ...]]:
        """Predicts the output of the module for the given input.

        This method is a wrapper around the `forward` method, which is used to 
        predict the output of the module for the given input. It is 
        particularly useful for making predictions on large datasets, as it 
        allows for the specification of a batch size for processing the input 
        data.

        Parameters
        ----------
        x : array-like
            The input data for which to predict the output. Should be an 
            array-like object with the same length as the input data.
        *args : Any
            Positional arguments for the input data. Should have the same 
            length as the input data.
        batch_size : int, optional
            The batch size for processing the input data. Defaults to 32.
        device : str, Device, optional
            The device on which to perform the prediction. If None, the model's
            device is used.
            Defaults to None.
        output_device : str, Device, optional
            The device on which to store the output. If None, the model's 
            device is used. Defaults to None.

        Returns
        -------
        Tensor or tuple of Tensor
            The output of the module for the given input data.
            Will match the output of the `forward` method. If the output is a 
            single tensor, it is returned as is. If the output is a tuple of 
            tensors, it is returned as a tuple.

        Examples
        --------
        To predict the output of a module for the given input data:
        
        >>> input_data = torch.randn(100, 3, 32, 32)
        >>> module = ConvolutionalNeuralNetwork(3, [], 1).build()
        >>> module.predict(input_data, batch_size=10)
        tensor([0.1, 0.2, 0.3, ...]) 

        Raises
        ------
        AssertionError
            If there are multiple input testora that have different lengths.
        
        """
        ...

### Writing Documentation for Classes

Beyond following the NumpyDoc style guide, Deeplay requires also the following sections:

- **Input**: This section should describe the input to the forward method. It should include the type of the input, the shape of the input, and any constraints on the input.

- **Output**: This section should describe the output of the forward method. It should include the type of the output, the shape of the output, and any constraints on the output.

- **Evaluation**: This section should describe the forward path of the class. Typically as code, but can also be in prose if the code is too complex.

The following describes how a class should be documented.

In [3]:
from deeplay import DeeplayModule

class ConvolutionalNeuralNetwork(DeeplayModule):
    """Convolutional Neural Network (CNN) component.

    This component is a convolutional neural network (CNN) that consists of 
    multiple convolutional blocks.
    Per default, there is no pooling applied between the blocks. The output of 
    the last block is not flattened.
    
    The default structure of the CNN is as follows:

    1. Conv2D(in_channels, hidden_channels[0], kernel_size=3, stride=1, 
              padding=1)
       ReLU
    2. Conv2D(hidden_channels[0], hidden_channels[1], kernel_size=3, stride=1, 
              padding=1)
       ReLU 
    ...
    n. Conv2D(hidden_channels[n-1], out_channels, kernel_size=3, stride=1, 
              padding=1)
       out_activation

    Parameters
    ----------
    in_channels: int or None
        Number of input features. If None, the input shape is inferred from the 
        first forward pass.
    hidden_channels: list[int]
        Number of hidden units in each layer except the last.
    out_channels: int
        Number of output features in the last layer.
    out_activation: Layer or type[nn.Module], optional
        Specification for the output activation of the last block. 
        (Default: nn.Identity)
    pool: template-like
        Specification for the pooling of the block. Is not applied to the first 
        block. (Default: nn.Identity)
        The pooling will be applied before the layer.
    
    Attributes
    ----------
    in_channels: int
        Number of input features.
    hidden_channels: list[int]
        Number of hidden units in each layer except the last.
    out_channels: int
        Number of output features in the last layer.
    blocks: LayerList
        List of blocks in the CNN.
    input: DeeplayModule
        first block in the CNN
    hidden: DeeplayModule
        all blocks except the last
    output: DeeplayModule
        last block in the CNN
    layer: LayerList
        List of layers in the CNN.
    activation: LayerList
        List of activation functions in the CNN.
    normalization: LayerList
        List of normalization functions in the CNN.
        
    Input
    -----
    x: float32
        (Any, in_channels, Any, Any)
        
    Output
    ------
    float32 : (Any, out_channels, Any, Any)

    Evaluation
    ----------
    >>> for block in cnn.blocks:
    >>>    x = block(x)
    >>> return x

    Examples
    --------
    >>> cnn = ConvolutionalNeuralNetwork(3, [16], 1, out_activation=nn.Sigmoid)
    >>> cnn.strided(2, apply_to_first=True) \
           .normalized(after_last_layer=False) \
           .build()
    ConvolutionalNeuralNetwork(
      (blocks): LayerList(
        (0): Conv2dBlock(
            (layer): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), 
                            padding=(1, 1))
            (activation): ReLU()
            (normalization): BatchNorm2d(16, eps=1e-05, momentum=0.1, 
                                         affine=True, track_running_stats=True)
        )
        (1): Conv2dBlock(
            (layer): Conv2d(16, 1, kernel_size=(3, 3), stride=(2, 2), 
                            padding=(1, 1))
            (activation): Sigmoid()
        )
    )

    """
    ...

### Writing Documentation for Modules

The following describes how a module should be documented.

In [4]:
"""Docstring for the example.py module.

Modules names should have short, all-lowercase names. The module name may
have underscores if this improves readability.

Every module should have a docstring at the very top of the file.  The
module's docstring may extend over multiple lines.  If your docstring does
extend over multiple lines, the closing three quotation marks must be on
a line by itself, preferably preceded by a blank line.

It is a good idea to provide an overview of the module here so that someone 
who reads the module's docstring does not have to read the entire file to
understand what the module does.

"""
...

Ellipsis

## Unit Testing

**All new features should be tested.** The tests should cover all possible code paths and should be as comprehensive as possible. The tests should be written using the `unittest` module in Python. The tests should be placed in the `tests` folder. The tests should be run using `unittest`. Not all tests follow our guidelines yet, but we are working on improving them.

**We aim to mirror the structure of the `deeplay` package in the `tests` package.** For example, `deeplay/external/layer.py` should have a corresponding `tests/external/test_layer.py` file. The name of the file should be the same as the module it tests, but with `test_` prepended. Note that when adding a folder, the `__init__.py` file should be added to the folder to make it a package.

**Each test file should contain one `unittest.TestCase` class per class or function being tested.** The test methods should be as descriptive as possible. For example, `test_forward` is not a good name for a test method. Instead, use `test_forward_with_valid_input` or `test_forward_with_invalid_input`. The test methods should be as independent as possible, but we value coverage over independence.

It is fine to use multiple subtests using `with self.subTest()` to test multiple inputs or edge cases. This is particularly useful when testing a method that has many possible inputs.

It is fine and preferred to use mocking where appropriate. For example, if a method calls an external API, the API call should be mocked. The `unittest.mock` module is very useful for this purpose.

Keep in mind to try to only test the the class or method itself, not the dependencies. For example, testing an application should focus on the training logic it implements, not the specificis of the architecture it uses.

### Testing Applications

Applications can be hard to test because they often require a lot of data and time to train to fully validate. This is not feasible, since we want to retain the unit test speed to within a few minutes. Here are some guidelines to follow when testing applications:

- Test both using build and create methods.
  - Validate that the forward pass works as expected.
  - Validate that the loss and metrics are correctly set.
  - Validate that the optimizer is correctly set.
    - This can be done by manually calling `app.configure_optimizer()` and checking that returned optimizer is correct the expected one.
    - Also, verify that the parameters of the optimizer are correctly set. This can
      be done by doing a forward pass, backward pass, and then checking that the optimizer's parameters have been updated.
      - If there are multiple optimizers with different parameters, ensure that the correct parameters are updated.

- Test `app.compute_loss` on a small input and verify that the loss is correct.

- Test `app.training_step` on a small input and verify that the loss is correct. Note that you might need to attach a trainer for this.

- Test training the application on a single epoch both using `app.fit` and `trainer.fit(app)`.
  - It's a good idea to check that the training history contains the correct keys.
  - Use as small a dataset and model as possible to keep the test fast. Turn off checkpointing and logging to speed up the test.

- Test that the application can be saved and loaded correctly.
  - Currently, we only guarantee `save_state_dict` and `load_state_dict` to work correctly. A good way to test is to save the state dict, create a new application, load the state dict, and then check that the new application has the same state as the old one.

- Test any application-specific methods, such as `app.detect`, `app.classify`, etc.

### Testing Models

Models are easier to test than applications because they are usually smaller and have fewer dependencies. We do not test if models can be trained.

- Test that the model, created with default arguments, can be created.

- Test that the model, created with default arguments, has the correct number of parameters.

- Test that the model, created with default arguments, can be saved and loaded using `save_state_dict` and `load_state_dict`.

- Test that a previously saved model state dict can be loaded into a new model.

- Test that the model, created with default arguments, has the expected hierarchical structure. This is mostly for forward compatibility.

- Test the forward pass with a small input and verify that the output is correct in terms of tensor shape.

- Test that the model can be created with non-default arguments and that the arguments are correctly set.

### Testing Components

Components are similar to models in terms of testing. They are generally smaller and have fewer dependencies. We do not test if components can be trained.

- Test that the component, created with default arguments, can be created.

- Test that the component, created with default arguments, can be reconstructed (by calling `.__construct__()`)

- Test that the component, created with default arguments, has the correct number of parameters.

- Test that the component, created with default arguments, can be saved and loaded using `.save_state_dict()` and `.load_state_dict()`.

- Test the forward pass with a small input and verify that the output is correct in terms of tensor shape.

- Test that the component can be created with non-default arguments and that the arguments are correctly set.

### Testing Blocks

Blocks are the most complex objects to test, as they are the building blocks of the library. They are generally larger and have more dependencies than models and components. We do not test if blocks can be trained.

- Test that the block, created with default arguments, can be created.

- Test that the block, created with default arguments, can be reconstructed (by calling `.__construct__()`).

- Test the various methods of the block (`.activated()`, `.normalized()`, etc.)

- Specifically test the `.multi()` method, ensure that the subblocks have the correct input/output features/channels.